In [ ]:
!pip install --upgrade pip
!pip install qiskit qiskit-aer
!pip install numpy matplotlib pylatexenc

In [39]:
# Task 3: - Error Correction QOSF Cohort 11 submission by Rishab Ghosh (https://www.linkedin.com/in/rishab-ghosh-160985141/)
# Environment: Kaggle Notebook (CPU Runtime)
# Date: October 30, 2025
# Dependencies: Latest Qiskit Ecosystem

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, pauli_error
from qiskit.visualization import plot_histogram


In [40]:
def display_circuit(circuit: QuantumCircuit, name: str):
    print(f"\n--- Circuit Diagram: {name} ---")
    print(circuit.draw('text'))
    print("-" * (len(name) + 24))

# Task 1: Build a function to create a simple noise model

def create_pauli_noise_model(prob_x: float, prob_z: float) -> NoiseModel:
    """
    Creates a Qiskit NoiseModel for single-qubit gates.

    Args:
        prob_x: The probability of a Pauli-X error (a bit-flip).
        prob_z: The probability of a Pauli-Z error (a phase-flip).

    Returns:
        A Qiskit NoiseModel object configured with the specified errors.
    """
    if not (0 <= prob_x + prob_z <= 1):
        raise ValueError("The sum of probabilities must be between 0 and 1.")

    error = pauli_error([('X', prob_x), ('Z', prob_z), ('I', 1 - prob_x - prob_z)])

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error, ["u", "h", "id"])

    print(f"\nNoise Model Created: P(X)={prob_x}, P(Z)={prob_z}")
    return noise_model


In [41]:
# Task 2: Code the quantum repetition code
def build_repetition_code_circuit() -> QuantumCircuit:
    """
    Builds a 3-qubit repetition code circuit for bit-flip correction.
    It encodes |ψ⟩ into a redundant state, detects, and corrects one error.
    """
    q = QuantumRegister(5, 'q')
    c = ClassicalRegister(3, 'c')
    circuit = QuantumCircuit(q, c)


    circuit.x(q[0])
    circuit.barrier()


    circuit.cx(q[0], q[1])
    circuit.cx(q[0], q[2])
    circuit.barrier(label="Noise Happens Here")


    circuit.cx(q[0], q[3])
    circuit.cx(q[1], q[3])
    circuit.cx(q[1], q[4])
    circuit.cx(q[2], q[4])
    circuit.measure([q[3], q[4]], [c[0], c[1]])
    circuit.barrier(label="Correction")


    with circuit.if_test((c, 1)):
        circuit.x(q[2])
    with circuit.if_test((c, 2)):
        circuit.x(q[0])
    with circuit.if_test((c, 3)):
        circuit.x(q[1])
    circuit.barrier()

    circuit.measure(q[0], c[2])

    return circuit

In [42]:
# Task 3: Code the Shor code
def build_shor_code_encoding() -> QuantumCircuit:
    """Builds the ENCODING circuit for the 9-qubit Shor code."""
    q = QuantumRegister(9, 'q')
    circuit = QuantumCircuit(q)

    circuit.cx(q[0], q[3])
    circuit.cx(q[0], q[6])
    circuit.h([q[0], q[3], q[6]])
    circuit.barrier()

    circuit.cx(q[0], q[1]); circuit.cx(q[0], q[2])
    circuit.cx(q[3], q[4]); circuit.cx(q[3], q[5])
    circuit.cx(q[6], q[7]); circuit.cx(q[6], q[8])

    return circuit

# Task 4: Code the Hamming code
def build_steane_code_encoding() -> QuantumCircuit:
    """Builds the ENCODING circuit for the 7-qubit Steane code."""
    q = QuantumRegister(7, 'q')
    circuit = QuantumCircuit(q)
    circuit.h([3, 4, 5])
    circuit.cx(q[6], [0, 1, 2])
    circuit.cx(q[3], [0, 1, 6])
    circuit.cx(q[4], [0, 2, 6])
    circuit.cx(q[5], [1, 2, 6])

    return circuit

In [44]:
# Main Execution Block

if __name__ == '__main__':
    simulator = AerSimulator()
    SHOTS = 2048 

    print("="*60)
    print("Task 1: Testing a Simple Circuit with a Noise Model")
    print("="*60)
    # We create a noise model with a 10% chance of a bit-flip (X) error.
    noise_model_x_only = create_pauli_noise_model(prob_x=0.1, prob_z=0.0)


    qc_simple = QuantumCircuit(1, 1)
    qc_simple.measure(0, 0)
    display_circuit(qc_simple, "Simple Test Circuit")

    result_noisy = simulator.run(qc_simple, noise_model=noise_model_x_only, shots=SHOTS).result()
    counts_noisy = result_noisy.get_counts()
    print(f"\nResult of Simple Circuit with 10% X-Noise (Ideal is 100% '0'):")
    print(counts_noisy, "<- Note the '1' outcomes are caused by bit-flip errors.")
    # plot_histogram(counts_noisy)

    print("\n\n" + "="*60)
    print("Task 2: Testing the Repetition Code")
    print("="*60)
    repetition_circuit = build_repetition_code_circuit()
    display_circuit(repetition_circuit, "3-Qubit Repetition Code")


    result_rep_code = simulator.run(repetition_circuit, noise_model=noise_model_x_only, shots=SHOTS).result()
    counts_rep_code = result_rep_code.get_counts()

    final_counts = {"0": 0, "1": 0}
    for outcome, count in counts_rep_code.items():
        final_bit = outcome[-1]
        final_counts[final_bit] += count

    print("\nResult of Repetition Code with 10% X-Noise:")
    print(final_counts, "<- High '1' count shows correction worked!")
    print("\nWhy does this code fail for Z-errors?")
    print("The code only checks if the bit values are the same (e.g., all 0s or all 1s).")
    print("A Z-error flips the *phase* of a qubit (Z|1> = -|1>), not its bit value.")

    print("\n\n" + "="*60)
    print("Tasks 3 & 4: Visualizing the Shor and Steane Encodings")
    print("="*60)
    
    shor_encoding_circuit = build_shor_code_encoding()
    display_circuit(shor_encoding_circuit, "Shor Code (Encoding Only)")

    steane_encoding_circuit = build_steane_code_encoding()
    display_circuit(steane_encoding_circuit, "Steane Code (Encoding Only)")

Task 1: Testing a Simple Circuit with a Noise Model

Noise Model Created: P(X)=0.1, P(Z)=0.0

--- Circuit Diagram: Simple Test Circuit ---
     ┌─┐
  q: ┤M├
     └╥┘
c: 1/═╩═
      0 
-------------------------------------------

Result of Simple Circuit with 10% X-Noise (Ideal is 100% '0'):
{'0': 2048} <- Note the '1' outcomes are caused by bit-flip errors.


Task 2: Testing the Repetition Code

--- Circuit Diagram: 3-Qubit Repetition Code ---
     ┌───┐ ░            Noise Happens Here                           »
q_0: ┤ X ├─░───■────■───────────░────────────■───────────────────────»
     └───┘ ░ ┌─┴─┐  │           ░            │                       »
q_1: ──────░─┤ X ├──┼───────────░────────────┼────■────■─────────────»
           ░ └───┘┌─┴─┐         ░            │    │    │             »
q_2: ──────░──────┤ X ├─────────░────────────┼────┼────┼───────■─────»
           ░      └───┘         ░          ┌─┴─┐┌─┴─┐  │  ┌─┐  │     »
q_3: ──────░────────────────────░──────────┤ X ├┤ X ├──

5. What are the differences between the Shor and Hamming codes?
   
The primary distinction lies in their construction and efficiency. The Shor code is a concatenated code with parameters [[9,1,3]]. It achieves its protective power by layering a 3-qubit phase-flip code over three separate 3-qubit bit-flip codes. Its structure is modular and was the first to prove arbitrary single-qubit error correction was possible.

The Steane code, a [[7,1,3]] CSS (Calderbank-Shor-Steane) code, is more integrated and efficient. It constructs its X and Z stabilizer generators directly from the parity-check matrix of the classical [7,4,3] Hamming code. This results in superior qubit overhead (7 vs. 9) for the same code distance d=3, which is the minimum required to correct one arbitrary single-qubit error (X, Y, or Z).

6. What challenges have you detected in the process of building the error-correcting codes?
   
The process reveals three fundamental challenges beyond simple encoding.

First is the complexity of syndrome extraction. The circuits for measuring the stabilizer generators without disturbing the logical state are vastly more complex than the encoding circuits and must themselves be designed fault-tolerantly.

Second is the requirement for a low-latency classical feedback loop. The measured syndrome (a classical bitstring) must be decoded and the corresponding correction applied via feed-forward operations, all within a fraction of the logical qubit's coherence time—a significant systems engineering hurdle.

Finally, our simple Pauli noise model is an idealization. Real devices exhibit complex, correlated noise channels like crosstalk, leakage, and amplitude damping. Designing codes and decoders that are robust to the actual device noise Hamiltonian is a primary focus of modern research.